# Benchmarking: numexpr

Timing a numexpr implementation of some basic mathematical operations.

## User input

In [1]:
import numpy as np

# storage
dtype = np.float64
storage_shape = (321, 321, 120)

# serialization
serialize = True
filename = "timings_daint.xlsx"
sheetname = "{} x {} x {}".format(*storage_shape)

## Data initialization

In [2]:
# install and import numexpr
is_installed = !pip list 2> /dev/null | grep 'numexpr'
if not is_installed:
    !pip install numexpr
import numexpr as ne

In [3]:
from copy import deepcopy

a = np.zeros(storage_shape, dtype=dtype)
a[...] = np.random.rand(*storage_shape)
a_dc = deepcopy(a)
b = np.zeros(storage_shape, dtype=dtype)
b[...] = np.random.rand(*storage_shape)
c = np.zeros(storage_shape, dtype=dtype)
c[...] = np.random.rand(*storage_shape)
d = np.zeros(storage_shape, dtype=dtype)

f = np.random.rand(1).item()

## Serialization setup

In [4]:
# install and import openpyxl
is_installed = !pip list 2> /dev/null | grep 'openpyxl'
if not is_installed:
    !pip install openpyxl==2.6
import openpyxl as xl

# install and import pandas
is_installed = !pip list 2> /dev/null | grep 'pandas'
if not is_installed:
    !pip install pandas
import pandas as pd

# install xlrd
is_installed = !pip list 2> /dev/null | grep 'xlrd'
if not is_installed:
    !pip install xlrd

In [5]:
# dataframe's indices and column
index = [
    "copychange", 
    "add", 
    "sub", 
    "mul", 
    "addsub",
    "fma",
    "sts_rk2_0",
    "sts_rk3ws_0"
]
column = "numexpr"
    
if serialize:
    # create the spreadsheet
    import os
    if not os.path.exists(filename):
        wb = xl.Workbook()
        wb.save(filename=filename)
    else:
        wb = xl.load_workbook(filename)
        
    # create an ExcelWriter object
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        writer.book = wb
        # writer.sheets = dict((ws.title, ws) for ws in wb.worksheets)
    
    if sheetname not in wb.sheetnames:
        # create an empty dataframe
        df = pd.DataFrame(data=[None,]*len(index), index=index, columns=[column,])
        df.to_excel(writer, sheet_name=sheetname)
    else:
        # load the dataframe
        df = pd.read_excel(writer, sheet_name=sheetname, index_col=0)
        
        # remove the sheet
        sheetid = wb.sheetnames.index(sheetname)
        wb.remove(wb.worksheets[sheetid])
else:
    # create the dataframe
    df = pd.DataFrame(data=[None,]*len(index), index=index, columns=[column,])

## Timing

In [6]:
# copychange
out = %timeit -o c = ne.evaluate("-b")

10.8 ms ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
# copychange with assignment
out = %timeit -o c[...] = ne.evaluate("-b")
df.at["copychange", column] = 1000 * out.average

20 ms ± 21.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
# add
out = %timeit -o c = ne.evaluate("a + b")

11.6 ms ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
# add with assignment
out = %timeit -o c[...] = ne.evaluate("a + b")
df.at["add", column] = 1000 * out.average

20.9 ms ± 81.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
# sub
out = %timeit -o c = ne.evaluate("a - b")

11.6 ms ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
# sub with assignment
out = %timeit -o c[...] = ne.evaluate("a - b")
df.at["sub", column] = 1000 * out.average

20.9 ms ± 77.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
# mul
out = %timeit -o c = ne.evaluate("a + b")

12.2 ms ± 63.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
# mul with assignment
out = %timeit -o c[...] = ne.evaluate("a + b")
df.at["mul", column] = 1000 * out.average

22.2 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
# addsub
out = %timeit -o d = ne.evaluate("a + b - c")

14.1 ms ± 31.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
# addsub with assignment
out = %timeit -o d[...] = ne.evaluate("a + b - c")
df.at["addsub", column] = 1000 * out.average

24 ms ± 66.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
# fma
out = %timeit -o c = ne.evaluate("a + f * b")

13.1 ms ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
# fma with assignment
out = %timeit -o c[...] = ne.evaluate("a + f * b")
df.at["fma", column] = 1000 * out.average

23.2 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
# sts_rk2_0
out = %timeit -o d = ne.evaluate("0.5 * (a + b + f * c)")

15.8 ms ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
# sts_rk2_0 with assignment
out = %timeit -o d[...] = ne.evaluate("0.5 * (a + b + f * c)")
df.at["sts_rk2_0", column] = 1000 * out.average

25.9 ms ± 70 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
# sts_rk3ws_0
out = %timeit -o d = ne.evaluate("2.0 * (a + b + f * c) / 3.0")

16.4 ms ± 35.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
# sts_rk3ws_0 with assignment
out = %timeit -o d[...] = ne.evaluate("2.0 * (a + b + f * c) / 3.0")
df.at["sts_rk3ws_0", column] = 1000 * out.average

26.5 ms ± 246 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Serialization

In [22]:
if serialize:
    df.to_excel(writer, sheet_name=sheetname)
    writer.save()

In [23]:
df

,numpy,numexpr
copy,9.449362,NaN
copychange,50.337060,20.007233
abs,49.926181,NaN
iabs,50.135189,NaN
add,55.094303,20.875209
iadd,15.295182,NaN
sub,54.904362,20.907313
isub,15.501360,NaN
mul,55.079714,22.223766
imul,18.344740,NaN
